<a href="https://colab.research.google.com/github/SZPiho/kfold-cross-validation-with-early-stopping/blob/main/kfold_cross_validation_with_early_stopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv("https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

In [2]:
df=pd.concat([df,pd.get_dummies(df['job'],prefix='job')],axis=1)
df.drop('job',axis=1,inplace=True)

In [3]:
df=pd.concat([df,pd.get_dummies(df['area'],prefix='arwea')],axis=1)
df.drop('area',axis=1,inplace=True)

In [4]:
med=df['income'].median()
df['income']=df['income'].fillna(med)

In [5]:
df.head(20)

,id,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,...,job_qp,job_qw,job_rn,job_sa,job_vv,job_zz,arwea_a,arwea_b,arwea_c,arwea_d
0,1,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,...,0,0,0,0,1,0,0,0,1,0
1,2,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,...,0,0,0,0,0,0,0,0,1,0
2,3,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,...,0,0,0,0,0,0,0,0,1,0
3,4,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,...,0,0,0,0,0,0,0,0,1,0
4,5,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,...,0,0,0,0,0,0,0,0,0,1
5,6,70854.0,40.400000,1,14.893343,87,20.340593,43,0.866142,0.673228,...,0,0,0,0,0,0,0,0,1,0
6,7,38726.0,30.975000,3,3.822477,33,9.480399,39,0.976378,0.874016,...,0,0,0,0,0,0,0,0,0,1
7,8,55162.0,26.966667,2,4.312097,17,29.219896,44,1.000000,0.724409,...,0,0,0,0,0,0,1,0,0,0
8,9,67311.0,32.383333,0,25.093772,169,10.927357,45,0.952756,0.681102,...,0,0,0,0,0,0,0,0,1,0
9,10,63344.0,38.233333,1,2.816034,3,21.915695,42,0.897638,0.724409,...,0,0,0,0,0,0,1,0,0,0


In [6]:
from scipy.stats import zscore

In [7]:
df['income']=zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

In [8]:
df.head(20)

,id,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,...,job_qp,job_qw,job_rn,job_sa,job_vv,job_zz,arwea_a,arwea_b,arwea_c,arwea_d
0,1,-0.607550,-0.664918,-0.208449,9.017895,-0.215764,11.738935,0.854321,0.885827,0.492126,...,0,0,0,0,1,0,0,0,1,0
1,2,0.338053,-0.207748,0.839031,7.766643,0.196869,6.805396,1.394432,0.874016,0.342520,...,0,0,0,0,0,0,0,0,1,0
2,3,-0.184205,1.127906,-0.208449,3.632069,-0.714362,13.671772,-0.495957,0.944882,0.724409,...,0,0,0,0,0,0,0,0,1,0
3,4,-0.526467,-0.440815,-0.208449,5.372942,-0.542432,4.333286,1.124377,0.889764,0.444882,...,0,0,0,0,0,0,0,0,1,0
4,5,-2.851675,1.638861,1.886511,3.822477,-0.473660,5.967121,-2.116291,0.744094,0.661417,...,0,0,0,0,0,0,0,0,0,1
5,6,1.382469,1.594040,-0.208449,14.893343,0.678274,20.340593,-0.766013,0.866142,0.673228,...,0,0,0,0,0,0,0,0,1,0
6,7,-1.817817,0.814162,1.886511,3.822477,-0.250150,9.480399,-1.846235,0.976378,0.874016,...,0,0,0,0,0,0,0,0,0,1
7,8,-0.180619,0.482489,0.839031,4.312097,-0.525239,29.219896,-0.495957,1.000000,0.724409,...,0,0,0,0,0,0,1,0,0,0
8,9,1.029549,0.930695,-1.255928,25.093772,2.088104,10.927357,-0.225902,0.952756,0.681102,...,0,0,0,0,0,0,0,0,1,0
9,10,0.634394,1.414758,-0.208449,2.816034,-0.765942,21.915695,-1.036069,0.897638,0.724409,...,0,0,0,0,0,0,1,0,0,0


In [9]:
x_columns=df.columns.drop('product').drop('id')
x=df[x_columns].values

In [10]:
dummies=pd.get_dummies(df['product'])
product=dummies.columns
y=dummies.values

In [11]:
y

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=uint8)

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import StratifiedKFold

In [13]:
kf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [14]:
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

In [15]:
oos_y=[]
oos_pred=[]
fold=0
for train,test in kf.split(x,df['product']):
  fold=fold+1
  print(f'Folf : #{fold}')
  x_train=x[train]
  y_train=y[train]
  x_test=x[test]
  y_test=y[test]
  model=Sequential()
  model.add(Dense(50,activation='relu',input_dim=x.shape[1]))
  model.add(Dense(25,activation='relu'))
  model.add(Dense(y.shape[1],activation='softmax'))
  model.compile(optimizer='adam',loss='categorical_crossentropy')
  monitor=EarlyStopping(monitor='val_loss',min_delta=1e-3,patience=5,verbose=2,mode='auto',restore_best_weights=True)
  model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=0,epochs=500)
  pred=model.predict(x_test)
  oos_y.append(y_test)
  pred=np.argmax(pred,axis=1)
  oos_pred.append(pred)
  y_compare=np.argmax(y_test,axis=1)
  score=accuracy_score(y_compare,pred)
  print(f'Fold score (accuracy):{score}')

oos_y=np.concatenate(oos_y)
oos_pred=np.concatenate(oos_pred)
oos_y_compare=np.argmax(oos_y,axis=1)
score=accuracy_score(oos_y_compare,oos_pred)
print(f"Final score (accuracy): {score}") 

Folf : #1
Restoring model weights from the end of the best epoch: 24.
Epoch 29: early stopping
Fold score (accuracy):0.6975
Folf : #2
Restoring model weights from the end of the best epoch: 21.
Epoch 26: early stopping
Fold score (accuracy):0.695
Folf : #3
Restoring model weights from the end of the best epoch: 20.
Epoch 25: early stopping
Fold score (accuracy):0.7175
Folf : #4
Restoring model weights from the end of the best epoch: 22.
Epoch 27: early stopping
Fold score (accuracy):0.7075
Folf : #5
Restoring model weights from the end of the best epoch: 31.
Epoch 36: early stopping
Fold score (accuracy):0.725
Final score (accuracy): 0.7085
